In [139]:
import os
import pandas as pd
from glob import glob
import matplotlib.image as mp_image
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [140]:
seed = 99 # go Aaron Judge!
np.random.seed(seed)

In [3]:
cd ../Data/data_overlay/overlayed_images_dermnet_ISIC_2018/

/home/ec2-user/SageMaker/teledermatologyAI_capstone


In [43]:
HOME = '/home/ec2-user/SageMaker/teledermatologyAI_capstone'
blended_dir = '/Data/data_overlay'
blended_p1 = '/overlayed_images_dermnet_ISIC_2018/'
blended_p2 = '/overlayed_images_part_three/'
blended_p3 = '/overlayed_images_part_two/'

In [110]:
p1 = glob(os.path.join(HOME + blended_dir + blended_p1, '*'+'.png'))
p2 = glob(os.path.join(HOME + blended_dir + blended_p2, '*'+'.png'))
p3 = glob(os.path.join(HOME + blended_dir + blended_p3, '*'+'.png'))
diverse_paths = pd.Series(p1 + p2 + p3, name  = 'F6_path')
diverse_ids = pd.Series([os.path.splitext(os.path.basename(i))[0] for i in blended_paths], name = 'image_id')

# paths = pd.

In [111]:
diverse_df = pd.concat([diverse_ids, diverse_paths], axis = 1)

In [100]:
pwd

'/home/ec2-user/SageMaker/teledermatologyAI_capstone'

In [113]:
data = pd.read_csv('/home/ec2-user/SageMaker/teledermatologyAI_capstone/full_data_final_not_diverse.csv', index_col = 0)
data.rename(columns = {'path': 'unaltered_path'}, inplace = True)

/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (7,11,12,13,14,15,16,17,18,19,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [141]:
new = data.merge(right = diverse_df, 
                 how = 'left', 
                 on = 'image_id')

# # some of the ids don't have a diverse representation for some reason
# # looks like this is fairly distributed given our splits of train/val/test
# new[(new.F6_path.isna()) & (new.split_10.isna() == False)].split_10.value_counts()

# # Also, looks like we may have not done this to stanford diverse, which is ok
# new[(new.F6_path.isna()) & (new.split_10.isna() == False)].source.value_counts()

# # For that reason, we need to subset the list of eligible IDs to be replaced with diverse images
# # - Don't want stanford data, it's already diverse
# # - Have to choose from data that has a diverse representation (path)
# # - Applying this sampling based on split 8, because it was the most successful run

ids_to_sample = new[(new.F6_path.isna() == False) & (new.split_8.isna() == False)]['image_id']
ids_to_replace = ids_to_sample.sample(frac = .33, 
                                      replace = False, 
                                      random_state = seed) 

In [151]:
new['path'] = np.where(new.image_id.isin(ids_to_replace), new.F6_path, new.unaltered_path)
new['overlayed'] = np.where(new.image_id.isin(ids_to_replace), 1, 0)
new.to_csv(f'{HOME}/full_data_final_diverse.csv')

In [183]:
reorder = ['image_id', 'diagnosis', 'age', 'sex', 'localization', 'source',
           'severity', 'unaltered_path', 'F6_path', 'path', 'duplicate', 
           'overlayed', 'label_0', 'label_4', 'label_5', 'label_6', 'split_0', 
           'split_1', 'split_2', 'split_3', 'split_4', 'split_5', 'split_6', 
           'split_7', 'split_8', 'split_9','split_10']
new = new[reorder]

In [185]:
new.to_csv(f'{HOME}/full_data_final_diverse.csv')

In [162]:
# new[new.overlayed == 1].split_9.value_counts()
# new[new.split_9.isna() == False].overlayed.value_counts()

0    23812
1     7818
Name: overlayed, dtype: int64

In [175]:
# tests = new.path.sample(3, random_state = seed)

In [182]:
# ipath = './Data/ISIC_2019/ISIC_2019_Training_Input/ISIC_0000000.jpg'
# ipath2 = '/home/ec2-user/SageMaker/teledermatologyAI_capstone/Data/data_overlay/overlayed_images_dermnet_ISIC_2018/ISIC_0028790.png'

# ipath2 = tests.iloc[0]
# image = mp_image.imread(ipath2)
# # imshow(image)
# fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (15, 15))
# axes.imshow(image)